# Lecture 03 (2/2): Integer Programming

# Team Formation
Let's consider an optimization problem where we want to form pairs of students for a team assignment. Imagine that we have 20 students in a class. Half of the students are in a master's program and half are in a PhD program. We want to form as many pairs as possible. Note,

* Some students already formed pairs and they have partners
* Some students prefer working alone
* We want to encourage master's students and PhD students to work together.

In [47]:
import pulp

students = [
    { "id": 0, "program": "master" },
    { "id": 1, "program": "master" },
    { "id": 2, "program": "master" },
    { "id": 3, "program": "master" },
    { "id": 4, "program": "master" },
    { "id": 5, "program": "master" },
    { "id": 6, "program": "master" },
    { "id": 7, "program": "master" },
    { "id": 8, "program": "master" },
    { "id": 9, "program": "master" },
    { "id": 10, "program": "phd" },
    { "id": 11, "program": "phd" },
    { "id": 12, "program": "phd" },
    { "id": 13, "program": "phd" },
    { "id": 14, "program": "phd" },
    { "id": 15, "program": "phd" },
    { "id": 16, "program": "phd" },
    { "id": 17, "program": "phd" },
    { "id": 18, "program": "phd" },
    { "id": 19, "program": "phd" },
]
sids = [s["id"] for s in students]
print(f"Students: {sids}")

from itertools import product
ss = list(product(sids, sids))
print(f"Pairs: {ss[:10]}...")

Students: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Pairs: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)]...


# Variables and Domains
We use a Python library called `pulp` to solve this integer programming problem. The first step is to define a problem. Then for the given problem, specify variables and their domains.

In [48]:
prob = pulp.LpProblem('TeamMaking-2', pulp.LpMaximize)
pair = pulp.LpVariable.dicts('pair', ss, cat='Binary')

# Constraints
We set a couple of constraints.

In [49]:
# Each student can only find one partner
for sid in sids:
    prob += pulp.lpSum([pair[sid, sid2] for sid2 in sids]) == 1

# The partnership needs to be symmetric
for _ss in ss:
    prob += pair[_ss[0], _ss[1]] - pair[_ss[1], _ss[0]] == 0

# Some students have already formed a pair
fixed_pairs = [(2, 3), (4, 5), (6, 7)]
for fp in fixed_pairs:
    prob += pair[fp[0], fp[1]] == 1

## Objective
In addition to the constraints, let's add objective. Let's say we want to encourage a PhD student and a non-PhD student to work together.

In [50]:
# Preference
def weight(sid1, sid2):
    reward = 0

    # It is nice if phd student and non-phd student work together
    program1 = students[sid1]['program']
    program2 = students[sid2]['program']
    if program1 != program2:
        reward += 1

    # It is nice if students work together with another student
    if sid1 != sid2:
        reward += 1

    return reward

In [51]:
prob += pulp.lpSum([pair[sid1, sid2] * weight(sid1, sid2) for sid1, sid2 in ss])

# Solve the problem

In [52]:
status = prob.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kotarohara/repo/Python/web-optimization/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/ce20cbf32d9d44c788e4c78b95c99959-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/ce20cbf32d9d44c788e4c78b95c99959-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 428 COLUMNS
At line 2792 RHS
At line 3216 BOUNDS
At line 3617 ENDATA
Problem MODEL has 423 rows, 400 columns and 1163 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 28 - 0.00 seconds
Cgl0004I processed model has 14 rows, 105 columns (105 integer (105 of which binary)) and 196 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solut

'Optimal'

In [53]:
for key in ss:
    if pair[key].value() > 0:
        print(pair[key])
        print(students[key[0]])
        print(students[key[1]])
        print()

pair_(0,_10)
{'id': 0, 'program': 'master'}
{'id': 10, 'program': 'phd'}

pair_(1,_13)
{'id': 1, 'program': 'master'}
{'id': 13, 'program': 'phd'}

pair_(2,_3)
{'id': 2, 'program': 'master'}
{'id': 3, 'program': 'master'}

pair_(3,_2)
{'id': 3, 'program': 'master'}
{'id': 2, 'program': 'master'}

pair_(4,_5)
{'id': 4, 'program': 'master'}
{'id': 5, 'program': 'master'}

pair_(5,_4)
{'id': 5, 'program': 'master'}
{'id': 4, 'program': 'master'}

pair_(6,_7)
{'id': 6, 'program': 'master'}
{'id': 7, 'program': 'master'}

pair_(7,_6)
{'id': 7, 'program': 'master'}
{'id': 6, 'program': 'master'}

pair_(8,_19)
{'id': 8, 'program': 'master'}
{'id': 19, 'program': 'phd'}

pair_(9,_12)
{'id': 9, 'program': 'master'}
{'id': 12, 'program': 'phd'}

pair_(10,_0)
{'id': 10, 'program': 'phd'}
{'id': 0, 'program': 'master'}

pair_(11,_18)
{'id': 11, 'program': 'phd'}
{'id': 18, 'program': 'phd'}

pair_(12,_9)
{'id': 12, 'program': 'phd'}
{'id': 9, 'program': 'master'}

pair_(13,_1)
{'id': 13, 'program'